## Import modules

In [1]:
import pickle
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorchtools
import glob as gl
import random
import os
import time
import itertools

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.datasets import make_regression
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import log_loss
from numpy import hstack
from numpy import vstack

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings(action='ignore')

## Data loading

In [2]:
def find_directory(foldername, filename = None, back_num = 0):
    cur = os.getcwd()
    for i in range(back_num):
        cur = os.path.abspath(os.path.join(cur, os.pardir))
    for folder in foldername:
        cur = os.path.join(cur, folder)
    if not os.path.exists(cur):
        os.makedirs(cur)
        print(f'{cur} created')
    if filename != None:
        cur = os.path.join(cur, filename)
    return cur

os.getcwd()
find_directory(back_num = 1, foldername = ['Dataset'], filename = 'bat_dict.pkl')

'C:\\Users\\User\\Desktop\\LIB\\CNN work\\Variable time length\\All dataset'

'C:\\Users\\User\\Desktop\\LIB\\CNN work\\Variable time length\\Dataset\\bat_dict.pkl'

In [3]:
bat_dict_add_1_2 = find_directory(back_num = 2, foldername = ['Dataset'], filename = 'bat_1_2_dict_VITQcQd.pkl')
with open(bat_dict_add_1_2, 'rb') as tf:
    bat_dict_1_2 = pickle.load(tf)

bat_sel_dict = bat_dict_1_2
max_time = 120

In [25]:
# Tensor dimension: (cell, cycle, time, variable)
num_cells = len(bat_sel_dict.keys())
regression_cycles = [30]

# variable = [Qc, I, V, T, Qd]
num_variables = len(bat_sel_dict['b1c0']['1']['1.0'].keys())

for reg in regression_cycles:
    #'''RNN 가변길이 다룰수 있게?? 고민필요'''
    globals()[f'x_tensor_{reg}'] = torch.zeros(num_cells, reg, max_time, num_variables)
    cell_index = 1
    for cell in bat_sel_dict.keys():
        for cycle in range(1, reg+1):
            for time in bat_sel_dict[cell][str(cycle)].keys():
    #             print(bat_sel_dict[cell][str(cycle)][str(time)])
                value_list = [i for i in bat_sel_dict[cell][str(cycle)][str(time)].values()]
    #             print(value_list)
                globals()[f'x_tensor_{reg}'][cell_index-1, cycle-1, int(float(time)*max_time/60), :] = torch.FloatTensor(value_list)

        cell_index += 1

    globals()[f'x_tensor_{reg}'].size()

num_variables = globals()[f'x_tensor_{reg}'].shape[3]
time_length = globals()[f'x_tensor_{reg}'].shape[2]
# (cycle_index-1, cycle-1, time-1, variable(Qc, I, V, T, Qd))
# x_tensor[123, 99, 30, :]

torch.Size([124, 30, 120, 5])

In [6]:
y_df_add = find_directory(back_num = 2, foldername = ['Dataset', 'y_df.csv'])
y_df = pd.read_csv(y_df_add)

# 2. Regression problem
# KO, KP, CL
y_cl_numpy = y_df['Cycle life'].to_numpy()
y_kp_numpy = y_df['kneepoints'].to_numpy()
y_ko_numpy = y_df['kneeonsets'].to_numpy()
# y_ko_numpy.shape

y_regression_numpy = y_ko_numpy

y_ko_tensor = torch.FloatTensor(y_ko_numpy)
y_ko_tensor.size()

torch.Size([124])

In [7]:
def setRandomSeed(random_seed=0):
    os.environ['PYTHONHASHSEED'] = str(random_seed)
    torch.manual_seed(random_seed) # torch 
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
    torch.backends.cudnn.deterministic = True # cudnn
    torch.backends.cudnn.benchmark = False # cudnn
    np.random.seed(random_seed) # numpy
    random.seed(random_seed) # random

In [9]:
globals()[f'x_tensor_{reg}'].shape

torch.Size([124, 80, 120, 5])

### Dataloading as 2D CNN's method

In [10]:
# y: no scaling
class dataPrep_RNN_CNN(Dataset):  
    def __init__(self, x_tensor, y_tensor, batch_size, scaler, purpose : str, test_split: float):
        setRandomSeed(random_seed = 100)
        self.xdata = torch.permute(x_tensor, (0, 3, 2, 1))
        self.ydata = y_tensor
        
        self.batch_size = batch_size
                
        self.xscaler = scaler()
        self.purpose = purpose
        
        self.test_split = test_split
        
        self.train_size, self.val_size, self.test_size = self.get_splits()
        
        # Tensor dimension: (cell, variable, time, cycle)
        # If problem emerges, check this part first.
        for i in range(len(self.xdata)):
            for j in range(self.xdata.shape[1]):
                temp = np.expand_dims(self.xscaler.fit_transform(self.xdata[i,j,:,:]), axis=0)
                if j ==0:
                    temp2 = temp
                else:
                    temp2 = np.vstack((temp2,temp))
            temp2 = np.expand_dims(temp2, axis=0)
            if i==0:
                self.xdata_scaled = temp2
            else:
                self.xdata_scaled = np.vstack((self.xdata_scaled, temp2))

        self.ydata_scaled = self.ydata
        
        self.x_train_scaled, self.x_test_scaled, self.y_train_scaled, self.y_test_scaled = train_test_split(self.xdata_scaled, 
                                                                                                            self.ydata_scaled, 
                                                                                                            test_size = self.test_size, 
                                                                                                            shuffle = True, 
                                                                                                            random_state=100)
        
        self.x_train_scaled, self.x_val_scaled, self.y_train_scaled, self.y_val_scaled = train_test_split(self.x_train_scaled,
                                                                                                         self.y_train_scaled,
                                                                                                         test_size = self.val_size, 
                                                                                                         shuffle = True, 
                                                                                                         random_state = 100)
        
        print(self.x_train_scaled.shape, self.x_val_scaled.shape, self.x_test_scaled.shape,
              self.y_train_scaled.shape, self.y_val_scaled.shape, self.y_test_scaled.shape)
        
        self.train = list(zip(self.x_train_scaled, self.y_train_scaled))
        self.val = list(zip(self.x_val_scaled, self.y_val_scaled))
        self.test = list(zip(self.x_test_scaled, self.y_test_scaled))
                
        self.train_dataloader = DataLoader(self.train, batch_size = self.batch_size, shuffle=True)
        self.val_dataloader = DataLoader(self.val, batch_size = self.batch_size, shuffle=True)
        self.test_dataloader = DataLoader(self.test, batch_size = self.batch_size, shuffle=True)
            
    # split the datasets into training and testing
    def get_splits(self):
        # test size
        test_size = int(self.test_split*len(self.xdata))
        # val size
        val_size = int(0.2*(len(self.xdata) - test_size))
        # train_size
        train_size = len(self.xdata) - val_size - test_size
        return train_size, val_size, test_size
    
    # Returns dataloaders for training and validation datasets
    def train_data(self):
        return self.train_dataloader, self.val_dataloader
    
    # Returns dataloader for test datasets
    def test_data(self):
        return self.test_dataloader
    
    def scaler(self, purpose):
#         if purpose != 'classification':
#             return self.xscaler, self.yscaler
#         else:
        return self.xscaler
    
    # finding length of x 
    def __len__(self):
        return len(self.X)
    
    # indexing rows for calling
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]


In [27]:
# reg_dataset = dataPrep1D(x_tensor, y_ko_tensor, batch_size = 16, scaler = MinMaxScaler, purpose = 'regression', test_split = 0.2)
# reg_train_dataloader, reg_val_dataloader = reg_dataset.train_data()
# reg_test_dataloader = reg_dataset.test_data()
# reg_xscaler = reg_dataset.scaler('regression')

regression_cycles = [30]
# regression_cycles = [100]


# x와 y가 1대1 대응이어야 dataloader를 만들 수 있어서 2D array 그대로 놓고 학습시 tensor transform 예정
for reg in regression_cycles:
    globals()[f'reg_dataset_{reg}'] = dataPrep_RNN_CNN(globals()[f'x_tensor_{reg}'], y_ko_tensor, batch_size = 4, scaler = MinMaxScaler, purpose =  'regression', test_split = 0.2)
    globals()[f'reg_train_dataloader_{reg}'], globals()[f'reg_val_dataloader_{reg}'] = globals()[f'reg_dataset_{reg}'].train_data()
    globals()[f'reg_test_dataloader_{reg}'] = globals()[f'reg_dataset_{reg}'].test_data()
    globals()[f'reg_xscaler_{reg}'] = globals()[f'reg_dataset_{reg}'].scaler('regression')

(80, 5, 120, 30) (20, 5, 120, 30) (24, 5, 120, 30) torch.Size([80]) torch.Size([20]) torch.Size([24])


## Define Model Class 

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [13]:
class RNNcell(nn.Module):
    def __init__(self, model, input_size, hidden_size, dropout = 0.1, bidirectional = False, num_layer=1):
        super(RNNcell, self).__init__()
        
        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
            
        self.model = model
        self.hidden_size = hidden_size
        self.num_layer = num_layer
        self.bidirectional = bidirectional
        
        self.D = 1 + self.bidirectional
        self.batch_size = 4
        
        #Batch size * cycles, timestep, input_size
        if self.model.endswith('RNN'):
            self.rnn = nn.RNN(input_size, hidden_size, num_layer, batch_first = True, dropout = dropout, bidirectional = bidirectional).to(self.device)
        elif self.model.endswith('LSTM'):
            self.rnn = nn.LSTM(input_size, hidden_size, num_layer, batch_first = True, dropout = dropout, bidirectional = bidirectional).to(self.device)
        elif self.model.endswith('GRU'):
            self.rnn = nn.GRU(input_size, hidden_size, num_layer, batch_first = True, dropout = dropout, bidirectional = bidirectional).to(self.device)
            
        for name, param in self.rnn.named_parameters():
            if name.startswith('weight'):
                nn.init.xavier_uniform(param)
            else:
                nn.init.normal(param)
        
    def forward(self, x):
        # x made from (cell * n_cy, timestep(seq_len), num_vars)
        self.batch_size = x.size(0)
        
        D = 1 + self.bidirectional
        #out: Containing output features h_t from the last layer of LSTM for each t(if bidirectional ==True: concat of forward and backward hidden states)
        #h_n: Containing final hidden state in the sequence
        h0 = torch.zeros(D*self.num_layer, self.batch_size, self.hidden_size).to(self.device)
        if self.model.endswith('LSTM'):
            c0 = torch.zeros(D*self.num_layer, self.batch_size, self.hidden_size).to(self.device)
            # c_n: Containing final cell state in the sequence
            out, (hn, cn) = self.rnn(x, (h0, c0))
        else:
            out, hn = self.rnn(x, h0.detach())
        return out, hn

In [14]:
class SelfAttention(nn.Module):
    def __init__(self, hidden_size, num_heads):
        super(SelfAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.head_size = hidden_size // num_heads
        
        self.query = nn.Linear(hidden_size, hidden_size)
        self.key = nn.Linear(hidden_size, hidden_size)
        self.value = nn.Linear(hidden_size, hidden_size)
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, inputs):
        # inputs shape: [batch_size, seq_len, hidden_size]
        batch_size, seq_len, hidden_size = inputs.size()
        
        # Compute query, key, and value matrices
        query = self.query(inputs).view(batch_size, seq_len, self.num_heads, self.head_size).transpose(1, 2)
        key = self.key(inputs).view(batch_size, seq_len, self.num_heads, self.head_size).transpose(1, 2)
        value = self.value(inputs).view(batch_size, seq_len, self.num_heads, self.head_size).transpose(1, 2)
        
        # Compute scaled dot-product attention scores
        # attention_scores: [batch_size, num_heads, seq_len, seq_len]
        attention_scores = torch.matmul(query, key.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.head_size, dtype=torch.float32))
        attention_probs = self.softmax(attention_scores)
        
        # Apply attention to the value matrix and concatenate heads
        # context: [batch_size, num_heads, seq_len, head_size]=>[batch_size, seq_len, num_heads, head_size] 
        # => [batch_size, seq_len, num_heads * head_size]
        context = torch.matmul(attention_probs, value).transpose(1, 2).contiguous().view(batch_size, seq_len, -1)
#         context = nn.Linear(hidden_size, 1)(context)
        return attention_probs, context

In [15]:
class RNN_TA_CA_1DCNN(nn.Module):
    def __init__(self, input_size, num_time, num_cycles, rnn1, bi1, hid1, nhead2, fil2, pool2, npool2, fsize2, psize2, mids, dr1 = 0.1, dr2 = 0.1, di2 = 1, st2 = 1, pad2 = 0):
        super(RNN_TA_CA_1DCNN, self).__init__()
        setRandomSeed()
        
        self.D1 = 1 + bi1
        
        self.rnn1 = RNNcell(rnn1, input_size, hid1, dr1, bi1)
        
        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
        
        # Overall parameters
        self.num_time = num_time
        self.num_cycles = num_cycles
        self.mids = mids
        
        # Pooling/Nonpooling parameters
        # Pooling/Nonpooling layers
        self.pool2 = pool2
        self.npool2 = npool2
        # Pooling size(Pooling layer)
        self.psize2 = psize2
        
        # Params for self-attention(ca)
        self.nh2 = nhead2
        
        # CNN hyperparameters
        self.fil2 = fil2
        self.fsize2 = fsize2
        self.di2 = di2
        self.st2 = st2
        # Padding for CNN
        self.pad2 = pad2
                
        # Critical values for the 1st CNN layer
        # Batch for different cells
        
        # Conv1d takes input dimension: [Batch_size, c_in(no. of in_channels )= hidden_size, l_in = num_cycles]
        self.in2 = self.D1*hid1
        self.out2 = self.fil2
        self.Lout = self.num_cycles
        
        # Conv_block1~P
        # Pooling layers
        for i in range(1, self.pool2+1):
#             print(f"i: {i}, out_channels: {self.out_channels}")
            globals()[f'self.conv_block{i}'] = nn.Sequential(
            nn.Conv1d(self.in2, self.out2, self.fsize2, self.st2, self.pad2),
            nn.BatchNorm1d(self.out2),
            nn.ReLU(),
            nn.MaxPool1d(self.psize2)
            ).to(self.device)

            self.in2 = self.out2
            self.out2 = self.out2*2
            self.Lout = int(int((self.Lout+2*self.pad2-self.di2*(self.fsize2-1)-1)/self.st2+1)/self.psize2)

        for j in range(self.pool2+1, self.pool2+self.npool2+1):
            globals()[f'self.conv_block{j}'] = nn.Sequential(
            nn.Conv1d(self.in2, self.out2, self.fsize2, self.st2, self.pad2),
            nn.BatchNorm1d(self.out2),
            nn.ReLU(),
            ).to(self.device)
            
            self.in2 = self.out2
            self.out2 = self.out2*2
            self.Lout = int((self.Lout+2*self.pad2-self.di2*(self.fsize2-1)-1)/self.st2+1)
        
        self.lin1 = nn.Linear(self.D1*hid1, 1)
        self.sm1 = nn.Softmax(dim = 2)
        self.relu = nn.ReLU()
        
        self.ca = SelfAttention(self.D1*hid1, self.nh2)
        
        self.fc = nn.Sequential(nn.Linear(self.in2*self.Lout, self.mids[0], bias=True), nn.ReLU(),
                       nn.Linear(self.mids[0], self.mids[1], bias = True), nn.ReLU(),
                       nn.Linear(self.mids[1], self.mids[2], bias = True))
        
        self.fc.apply(self.init_weights)
        
    def init_weights(self, m):
        self.m = m
        if type(self.m) == nn.Linear:
            nn.init.xavier_uniform_(self.m.weight)
        
    def forward(self, x):
        # Original: Batch size, num_vars, timestep, cycles=>Batch size * cycles, timestep, input_size
        x = torch.reshape(x, (x.size(0)*x.size(3), x.size(2), x.size(1))).to(device)
#         print("Batch size * cycles, timestep, input_size: ", x.size())
        out1, hn1 = self.rnn1(x)
        
        # hn의 batch size가 n_cy*k로 설정
        cell_batch_size = int(hn1.size(1)/self.num_cycles)

        # Reshape for input
        # out1: (batch_size = num_cell*num_cycle, timestep, D1*hid1)
        # outs : (batch_size(num_cell), num_cycle, timestep, D1*hid1)
        outs1 = torch.reshape(out1, (cell_batch_size, -1, out1.size(1), out1.size(2)))
        # ta: (batch_size, num_cycle, timestep, 1(D1*hid1->1))
        ta = self.sm1(self.lin1(outs1))
        # ta_outs: (batch_size, num_cycle, timestep, D1*hid1)=>(batch_size, D1*hid1, num_cycle, timestep)
        ta_outs = torch.permute(ta*outs1, (0, 3, 1, 2))
        # ct_vec: (batch_size, D1*hid1, num_cycle)
        ct_vec = torch.sum(ta_outs, -1)
        # ct_vec shape: [batch_size, num_cycle, D1*hid1(hidden_size)]
        # hidden_size = num_heads * head_size
        # ct_vec transform back: [batch_size, num_heads, num_cycle, head_size]=>[batch_size, num_cycle, num_heads, head_size] 
        # => [batch_size, num_cycle, num_heads * head_size(hidden_size)]
        # ca: matmul(query[batch_size, num_heads, num_cycle, head_size], key^T[batch_size, num_heads, head_size, num_cycle])
        # ca shape: [batch_size, num_heads, num_cycle, num_cycle]
        ca, ct_vec = self.ca(ct_vec.transpose(2, 1))
#         print(f"After self attention: ca.size: {ca.size()}, ct_vec.size: {ct_vec.size()}")
        # ct_vec shape: [batch_size, D1*hid1, num_cycle]
        ct_vec = ct_vec.transpose(2, 1)
        
        
        # input for 1d cnn: (batch_size, D1*hid1, num_cycle)
        for i in range(1, self.pool2+self.npool2+1):
            ct_vec = globals()[f'self.conv_block{i}'](ct_vec)
        # ct_vec from final convolutional layer: (batch_size, self.in2*self.Lout)
        ct_vec = ct_vec.view(ct_vec.size(0), -1)
        # final_out: (batch_size, 1)
        final_out = self.fc(ct_vec)
        
        return final_out.squeeze(), ta.squeeze(), ca

In [16]:
# Function for saving and loading of training history
def save_data(D3_array, filename):
    with open(filename,"wb") as dat_:
        pickle.dump(D3_array,dat_)
        
def load_data(filename):
    with open(filename,"rb") as ld:
        x_temp = pickle.load(ld)
    return x_temp

# Early stopping class
class EarlyStopping:
    """주어진 patience 이후로 validation loss가 개선되지 않으면 학습을 조기 중지"""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        """
        Args:
            patience (int): validation loss가 개선된 후 기다리는 기간
                            Default: 7
            verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
                            Default: False
            delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
                            Default: 0
            path (str): checkpoint저장 경로
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.best_epoch = 1
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, epoch, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.best_model = self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
#             print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.best_epoch = epoch
            self.best_model = self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''validation loss가 감소하면 모델을 저장한다.'''
        if self.verbose:
            print(f'At epoch {self.best_epoch}: Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss
        return model

In [17]:
class train:
    def __init__(self, model: nn.Module, train_dataloader, val_dataloader, test_dataloader, epoch: int, learning_rate=0.01, patience = 5, verbose = False):
        super().__init__()
        #   Reprocudtion
        setRandomSeed(100)
        random_seed = 100
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(random_seed)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
        os.environ['PYTHONHASHSEED'] = str(random_seed)
        torch.manual_seed(random_seed)
        np.random.seed(random_seed)

        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
        
        self.model = model.to(self.device)
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader
        self.test_dataloader = test_dataloader
        self.epoch = epoch
        self.learning_rate = learning_rate
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr = self.learning_rate)
        self.early_stopping = EarlyStopping(patience = patience, verbose = verbose)

        self.criterion = nn.MSELoss()
        self.name = ['train_rmse', 'val_rmse', 'test_rmse']
        
    def EvalModel(self):
        EvalPredictions, EvalActuals = list(), list()
        TestPredictions, TestActuals = list(), list()
        self.model.eval()
        with torch.no_grad():
            for n, (inputs, targets) in enumerate(self.val_dataloader):
                inputs = inputs.float().to(self.device)
                targets = targets.float().to(self.device)

                yhat, ta, ca = self.model(inputs)

                if self.device !='cpu':
                    yhat = yhat.to('cpu')
                    targets = targets.to('cpu')    

                yhat = yhat.detach().numpy()
                actual = targets.numpy()
                EvalPredictions.append(yhat)
                EvalActuals.append(actual)

            EvalPredictions, EvalActuals = vstack(EvalPredictions), vstack(EvalActuals)
            val_rmse = mean_squared_error(EvalActuals, EvalPredictions, squared = False)
            
            for n, (inputs, targets) in enumerate(self.test_dataloader):
                inputs = inputs.float().to(self.device)
                targets = targets.float().to(self.device)

                yhat, ta, ca = self.model(inputs)

                if self.device !='cpu':
                    yhat = yhat.to('cpu')
                    targets = targets.to('cpu')    

                yhat = yhat.detach().numpy()
                actual = targets.numpy()
                TestPredictions.append(yhat)
                TestActuals.append(actual)

            TestPredictions, TestActuals = vstack(TestPredictions), vstack(TestActuals)
            test_rmse = mean_squared_error(TestActuals, TestPredictions, squared = False)
            
            return val_rmse, test_rmse

    def TrainModel(self):
        loss_history = []
        self.model.train()
        for i in range(self.epoch):
            predictions, actuals = list(), list()
            for n, (inputs, targets) in enumerate(self.train_dataloader):
                self.model.train()
                inputs = inputs.float().to(self.device)
                targets = targets.float().to(self.device)

                self.optimizer.zero_grad()

                yhat, ta, ca = self.model(inputs)
                    
                loss = self.criterion(yhat, targets)
                loss.backward()
                self.optimizer.step()
                
                if self.device !='cpu':
                    yhat = yhat.to('cpu')
                    targets = targets.to('cpu')
                    
                yhat = yhat.detach().numpy()
                predictions.append(yhat)
                
                targets = targets.numpy()
                actuals.append(targets)

            # evaluation of entire train data
            predictions, actuals = vstack(predictions), vstack(actuals)
            
            train_rmse = mean_squared_error(actuals, predictions, squared = False)

            # evaluation on validation data
            val_rmse, test_rmse = self.EvalModel()
            early_factor = val_rmse
            
            # training loss history
            loss_history.append([train_rmse, val_rmse, test_rmse])

            self.early_stopping(i+1, early_factor, self.model)
            
            if self.early_stopping.early_stop:
                print("Early stopping at best epoch: ", self.early_stopping.best_epoch)
                self.best_loss = loss_history[self.early_stopping.best_epoch-1]
                loss_history = loss_history[:self.early_stopping.best_epoch]
                
                return self.model, self.best_loss, pd.DataFrame(loss_history, columns = self.name), ta, ca
            
        print(f"Ends at final epoch {self.epoch}")
        print(f"Best epoch: {self.early_stopping.best_epoch}")
        self.best_loss = loss_history[self.early_stopping.best_epoch-1]
        loss_history = loss_history[:self.early_stopping.best_epoch]
        
        return self.model, self.best_loss, pd.DataFrame(loss_history, columns = self.name), ta, ca
        
    def predict(self, data: torch.Tensor):
        return self.model(data)
    
    def train_val_test_plot_rmse(self, num_cycles, modelname, n_ep, patience, rnn1, hid1, fil2, pool2, npool2, fsize2, psize2, lr):
        dataloaders = [self.train_dataloader, self.val_dataloader, self.test_dataloader]

        with torch.no_grad():
            model.eval()

            model_rmse = list()
            dset_index = 0
            for dataloader in dataloaders:
                if dset_index ==0:
                    dset = 'train'
                elif dset_index == 1:
                    dset = 'val'
                else:
                    dset = 'test'

                figadd = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'train_history'], 
                                             filename = f'{rnn1}_hidden_{hid1}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}.jpg')

                predictions, actuals = list(), list()
                for i, (test_input, test_target) in enumerate(dataloader):
                    test_input = test_input.float().to(self.device)
                    test_target = test_target.float().to(self.device)
                    test_yhat, test_ta = model(test_input)

                    if self.device != 'cpu':
                        test_yhat = test_yhat.to('cpu')
                        test_target = test_target.to('cpu')

                    test_yhat = test_yhat.detach().tolist()
                    predictions += test_yhat
                    test_target = test_target.tolist()
                    actuals += test_target

                pred_acc, targ_acc = np.array(predictions), np.array(actuals)

                rmse = mean_squared_error(actuals, predictions, squared = False)
                model_rmse.append(rmse)

                plt.figure(figsize=[10,4])
                plt.rcParams["font.size"] = "12"
                plt.plot(pred_acc, label='Prediction')
                plt.plot(range(len(targ_acc)), targ_acc, label = 'Actual')
                plt.xticks(range(0, len(targ_acc) + 1, int(len(targ_acc)/5)))
                plt.title(f"{rnn1}_hid_{hid1}_nfil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}: RMSE={rmse}")
                plt.xlabel('cell')
                plt.ylabel('Lifespan(cycle)')
                plt.legend()
                plt.savefig(figadd)
                plt.close()

                dset_index += 1

        return model_rmse

## Implementation

In [18]:
def history_state_dict_add(num_cycles, modelname, n_ep, patience, rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr):
    history_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'train_history'], 
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}.pkl')
    state_dict_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'model'], 
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_state_dict.pth')
    ta_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'ta'],
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_ta.wb')
    ca_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'ca'],
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_ta.wb')
    return history_add, state_dict_add, ta_add, ca_add

In [28]:
# Hyperparameters
num_vars = globals()[f'x_tensor_{reg}'].size()[3]

modelname = 'RNN_TA_CA_1DCNN'
rnns = ['LSTM', 'GRU', 'RNN']
hids = [3, 5, 7]
num_time = max_time
num_cycless = [30]
ep_pats = [[3000, 500], [1000, 10], [2000, 20]]
lrs = [1e-4, 1e-3, 1e-2]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]

nh2s = [1]

import time

for [n_ep, patience] in ep_pats:
    for num_cycles in num_cycless:
        print(f"num_cycle = {num_cycles}, epoch = {n_ep}, patience = {patience}")
        trdl = globals()[f'reg_train_dataloader_{num_cycles}']
        vdl = globals()[f'reg_val_dataloader_{num_cycles}']
        tedl = globals()[f'reg_test_dataloader_{num_cycles}']
        for rnn1, hid1 in itertools.product(rnns, hids):
            bi1 = False

            if rnn1.startswith('Bi'):
                bi1 = True
            for pool2, npool2 in itertools.product(pools, npools):
                print(f'Pooling: {pool2} layers, Nonpooling: {npool2} layers, split by {60/max_time} min')
                for fil2, lr in itertools.product(num_fils, lrs):
                    for nh2 in nh2s:
                        print(f"rnn = {rnn1}, bi = {bi1}, hid per head = {hid1}, num_heads = {nh2}, num_fil = {fil2}, lr = {lr}")

                        all_hid1 = hid1 * nh2

                        start = time.time()

                        # Construct CNN
                        model = globals()[modelname](num_vars, num_time, num_cycles, rnn1, bi1, all_hid1, nh2, fil2, pool2, npool2, fsize2, psize2, mids).to(device)

                        # Train
                        model_train = train(model, trdl, vdl, tedl, n_ep, lr, patience, verbose = False)
                        best_model, best_loss, history, ta, ca = model_train.TrainModel()

                        print('best_loss = '+ str(best_loss))

                        history_add, state_dict_add, ta_add, ca_add = history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                                    rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr)

                        # saving best_epoch, loss history, ta score
                        save_data(history, history_add)
                        save_data(best_model.state_dict(), state_dict_add)
                        save_data(ta, ta_add)
                        save_data(ca, ca_add)

                        print("time: ", time.time()-start)
                        print("\n\n")

num_cycle = 30, epoch = 3000, patience = 500
Pooling: 1 layers, Nonpooling: 1 layers, split by 0.5 min
rnn = LSTM, bi = False, hid per head = 3, num_heads = 1, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  1426
best_loss = [151.41006, 106.46637, 101.80232]
C:\Users\User\Desktop\LIB\CNN work\Variable time length\All dataset\30 cycles\Depth Test_col_3000_500\RNN_TA_CA_1DCNN\LSTM_1D CNN\train_history created
C:\Users\User\Desktop\LIB\CNN work\Variable time length\All dataset\30 cycles\Depth Test_col_3000_500\RNN_TA_CA_1DCNN\LSTM_1D CNN\model created
C:\Users\User\Desktop\LIB\CNN work\Variable time length\All dataset\30 cycles\Depth Test_col_3000_500\RNN_TA_CA_1DCNN\LSTM_1D CNN\ta created
C:\Users\User\Desktop\LIB\CNN work\Variable time length\All dataset\30 cycles\Depth Test_col_3000_500\RNN_TA_CA_1DCNN\LSTM_1D CNN\ca created
time:  195.45662903785706



rnn = LSTM, bi = False, hid per head = 3, num_heads = 1, num_fil = 3, lr = 0.001
Early stopping at best epoch:  382
best_loss 

Ends at final epoch 3000
Best epoch: 2955
best_loss = [288.51407, 200.0489, 168.64072]
time:  333.41724586486816



rnn = LSTM, bi = False, hid per head = 5, num_heads = 1, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  1649
best_loss = [179.41624, 106.30728, 94.354095]
time:  240.76486659049988



rnn = LSTM, bi = False, hid per head = 5, num_heads = 1, num_fil = 5, lr = 0.001
Early stopping at best epoch:  403
best_loss = [125.77906, 132.95775, 78.19049]
time:  100.99590349197388



rnn = LSTM, bi = False, hid per head = 5, num_heads = 1, num_fil = 5, lr = 0.01
Early stopping at best epoch:  180
best_loss = [129.35707, 93.20578, 79.109474]
time:  76.05392742156982



rnn = LSTM, bi = False, hid per head = 5, num_heads = 1, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  649
best_loss = [204.14761, 120.7278, 139.84375]
time:  131.1121895313263



rnn = LSTM, bi = False, hid per head = 5, num_heads = 1, num_fil = 7, lr = 0.001
Early stopping at best epoch:  185
best_lo

Early stopping at best epoch:  197
best_loss = [129.5583, 85.08969, 72.42864]
time:  80.02998113632202



Pooling: 1 layers, Nonpooling: 2 layers, split by 0.5 min
rnn = LSTM, bi = False, hid per head = 7, num_heads = 1, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  1986
best_loss = [574.5814, 527.3776, 522.5423]
time:  288.18280267715454



rnn = LSTM, bi = False, hid per head = 7, num_heads = 1, num_fil = 3, lr = 0.001
Ends at final epoch 3000
Best epoch: 2780
best_loss = [534.49554, 477.62067, 470.5147]
time:  346.5142457485199



rnn = LSTM, bi = False, hid per head = 7, num_heads = 1, num_fil = 3, lr = 0.01
Ends at final epoch 3000
Best epoch: 2955
best_loss = [288.55716, 200.06725, 168.7071]
time:  347.4697711467743



rnn = LSTM, bi = False, hid per head = 7, num_heads = 1, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  1054
best_loss = [210.02187, 120.27655, 123.363556]
time:  180.97682785987854



rnn = LSTM, bi = False, hid per head = 7, num_heads = 1, num_

Early stopping at best epoch:  212
best_loss = [117.04137, 85.173965, 81.79281]
time:  78.67794585227966



rnn = GRU, bi = False, hid per head = 3, num_heads = 1, num_fil = 5, lr = 0.0001
Ends at final epoch 3000
Best epoch: 2818
best_loss = [126.102905, 100.83184, 65.066444]
time:  331.36693358421326



rnn = GRU, bi = False, hid per head = 3, num_heads = 1, num_fil = 5, lr = 0.001
Early stopping at best epoch:  463
best_loss = [122.79599, 97.0777, 81.61778]
time:  106.28033375740051



rnn = GRU, bi = False, hid per head = 3, num_heads = 1, num_fil = 5, lr = 0.01
Early stopping at best epoch:  163
best_loss = [113.52819, 125.89511, 99.19719]
time:  73.50604128837585



rnn = GRU, bi = False, hid per head = 3, num_heads = 1, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  1125
best_loss = [205.70001, 142.85101, 110.502144]
time:  185.7243390083313



rnn = GRU, bi = False, hid per head = 3, num_heads = 1, num_fil = 7, lr = 0.001
Early stopping at best epoch:  622
best_loss = 

Early stopping at best epoch:  1986
best_loss = [573.9582, 526.6827, 521.8567]
time:  280.11740589141846



rnn = GRU, bi = False, hid per head = 5, num_heads = 1, num_fil = 3, lr = 0.001
Ends at final epoch 3000
Best epoch: 2780
best_loss = [533.89655, 476.93475, 469.83325]
time:  337.14766454696655



rnn = GRU, bi = False, hid per head = 5, num_heads = 1, num_fil = 3, lr = 0.01
Ends at final epoch 3000
Best epoch: 2955
best_loss = [288.51108, 200.04762, 168.6361]
time:  337.90387177467346



rnn = GRU, bi = False, hid per head = 5, num_heads = 1, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  803
best_loss = [195.61902, 106.57408, 97.79652]
time:  148.14792323112488



rnn = GRU, bi = False, hid per head = 5, num_heads = 1, num_fil = 5, lr = 0.001
Early stopping at best epoch:  671
best_loss = [120.01563, 89.750374, 107.21877]
time:  131.4440314769745



rnn = GRU, bi = False, hid per head = 5, num_heads = 1, num_fil = 5, lr = 0.01
Early stopping at best epoch:  258
best_lo

...

Early stopping at best epoch:  51
best_loss = [169.24821, 105.198074, 82.18971]
time:  6.178996562957764



rnn = GRU, bi = False, hid per head = 3, num_heads = 1, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  29
best_loss = [572.6247, 534.8207, 523.469]
time:  4.465993404388428



rnn = GRU, bi = False, hid per head = 3, num_heads = 1, num_fil = 5, lr = 0.001
Early stopping at best epoch:  52
best_loss = [283.9052, 188.7282, 187.17284]
time:  6.555152893066406



rnn = GRU, bi = False, hid per head = 3, num_heads = 1, num_fil = 5, lr = 0.01
Early stopping at best epoch:  17
best_loss = [222.47107, 140.2832, 117.06996]
time:  3.365996837615967



rnn = GRU, bi = False, hid per head = 3, num_heads = 1, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  285
best_loss = [258.8164, 155.82222, 142.51419]
time:  29.379435539245605



rnn = GRU, bi = False, hid per head = 3, num_heads = 1, num_fil = 7, lr = 0.001
Early stopping at best epoch:  35
best_loss = [225.21667, 171.177

Early stopping at best epoch:  24
best_loss = [570.84955, 537.96686, 527.17084]
time:  3.3299970626831055



rnn = GRU, bi = False, hid per head = 7, num_heads = 1, num_fil = 3, lr = 0.001
Early stopping at best epoch:  70
best_loss = [194.16734, 152.4999, 132.29877]
time:  6.804995775222778



rnn = GRU, bi = False, hid per head = 7, num_heads = 1, num_fil = 3, lr = 0.01
Early stopping at best epoch:  12
best_loss = [197.13446, 120.97446, 84.70524]
time:  2.4209980964660645



rnn = GRU, bi = False, hid per head = 7, num_heads = 1, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  24
best_loss = [571.45844, 538.54016, 527.73676]
time:  3.3350331783294678



rnn = GRU, bi = False, hid per head = 7, num_heads = 1, num_fil = 5, lr = 0.001
Early stopping at best epoch:  122
best_loss = [202.68059, 156.45433, 141.82999]
time:  10.732107400894165



rnn = GRU, bi = False, hid per head = 7, num_heads = 1, num_fil = 5, lr = 0.01
Early stopping at best epoch:  16
best_loss = [226.39618, 

Early stopping at best epoch:  58
best_loss = [228.32047, 143.7485, 128.85843]
time:  5.745650768280029



rnn = RNN, bi = False, hid per head = 3, num_heads = 1, num_fil = 5, lr = 0.01
Early stopping at best epoch:  92
best_loss = [145.62984, 108.583305, 75.83012]
time:  8.280149221420288



rnn = RNN, bi = False, hid per head = 3, num_heads = 1, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  251
best_loss = [305.1626, 195.84698, 225.25244]
time:  21.132238388061523



rnn = RNN, bi = False, hid per head = 3, num_heads = 1, num_fil = 7, lr = 0.001
Early stopping at best epoch:  58
best_loss = [261.97382, 135.30643, 181.1369]
time:  6.126964092254639



rnn = RNN, bi = False, hid per head = 3, num_heads = 1, num_fil = 7, lr = 0.01
Early stopping at best epoch:  21
best_loss = [228.18268, 140.25287, 127.614655]
time:  3.2114977836608887



Pooling: 1 layers, Nonpooling: 2 layers, split by 0.5 min
rnn = RNN, bi = False, hid per head = 3, num_heads = 1, num_fil = 3, lr = 0.0001
E

Early stopping at best epoch:  27
best_loss = [190.29425, 129.40959, 92.06856]
time:  3.8057241439819336



rnn = RNN, bi = False, hid per head = 5, num_heads = 1, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  224
best_loss = [271.82446, 214.98677, 210.10974]
time:  19.957979917526245



rnn = RNN, bi = False, hid per head = 5, num_heads = 1, num_fil = 5, lr = 0.001
Early stopping at best epoch:  25
best_loss = [277.2298, 186.85274, 204.23082]
time:  3.7024803161621094



rnn = RNN, bi = False, hid per head = 5, num_heads = 1, num_fil = 5, lr = 0.01
Early stopping at best epoch:  79
best_loss = [135.97253, 87.32585, 78.79808]
time:  8.082566022872925



rnn = RNN, bi = False, hid per head = 5, num_heads = 1, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  161
best_loss = [316.48505, 245.25554, 253.30069]
time:  15.05842137336731



rnn = RNN, bi = False, hid per head = 5, num_heads = 1, num_fil = 7, lr = 0.001
Early stopping at best epoch:  70
best_loss = [203.31442, 

Early stopping at best epoch:  24
best_loss = [570.1989, 537.25757, 526.18805]
time:  3.5872342586517334



rnn = RNN, bi = False, hid per head = 7, num_heads = 1, num_fil = 3, lr = 0.001
Early stopping at best epoch:  61
best_loss = [303.15225, 224.35356, 206.57845]
time:  6.706839084625244



rnn = RNN, bi = False, hid per head = 7, num_heads = 1, num_fil = 3, lr = 0.01
Early stopping at best epoch:  12
best_loss = [193.9353, 110.509224, 139.41531]
time:  2.6503939628601074



rnn = RNN, bi = False, hid per head = 7, num_heads = 1, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  29
best_loss = [574.1127, 536.6894, 525.4035]
time:  4.076391220092773



rnn = RNN, bi = False, hid per head = 7, num_heads = 1, num_fil = 5, lr = 0.001
Early stopping at best epoch:  99
best_loss = [248.10342, 150.36325, 160.4949]
time:  9.985088348388672



rnn = RNN, bi = False, hid per head = 7, num_heads = 1, num_fil = 5, lr = 0.01
Early stopping at best epoch:  52
best_loss = [149.34572, 74.431

In [ ]:
modelname = 'RNN_TA_CA_1DCNN'
rnns = ['LSTM', 'GRU', 'RNN']
hids = [3, 5, 7]
num_time = max_time
num_cycles = 100
ep_pats = [[1000, 10], [2000, 20], [3000, 500]]
lrs = [1e-4, 1e-3, 1e-2]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]

nh2s = [2, 3, 4, 5]

for [n_ep, patience] in ep_pats:
    for nh2 in nh2s:
        for rnn1 in rnns:
            df = pd.DataFrame(columns = ['train rmse', 'val rmse', 'test rmse'])
            bi1 = False
            if rnn1.startswith('Bi'):
                bi1 = True
            for hid1 in hids:
                for pool2, npool2 in itertools.product(pools, npools):
                    for fil2, lr in itertools.product(num_fils, lrs):
                        history_add, state_dict_add, ta_add, ca_add = history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                                    rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr)

                        history = load_data(history_add)

                        df.loc[f'{rnn1}_{hid1}_{fil2}_{pool2}_{npool2}_{fsize2}_{lr}'] = [history.iloc[-1, 0], history.iloc[-1, 1], history.iloc[-1, 2]]

        print(n_ep, patience, rnn1)
        df
        df_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname], 
                        filename = f'{modelname}_{rnn1}_{nh2}.csv')
        df.to_csv(df_add)